This script analyzes the effect of the quantum modes on the classical geometry and can be used both for setups with and without backreaction. The bilinears are used to calculate the effect of the quantum modes on the classical stress energy tensor. As is, it works correclty only with fixed rmax. With minor adaptations it can work correctly with infalling rmax, as well. In its current form and for infalling rmax, it cannot plot the correct radial domain, that shrinks, but only the original one.

In [ ]:
# loads the needed packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpecialFunctions
using FunctionZeros
using Interpolations

using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.0_b0.0_c1.0_rmax30.0_tmax10.0_cfl0.0625_sigma0.0_overMp2_0.9988967160535545_reg_true_backreact_true_rcut20.0_mPV1.0_dk_denom_30_kmax10.0_lmax30.0"
your_dir = dir*par

Set the number of points Nr of the simulation, load the radial grid (fixed), read all the h5 files with specific prefixes, and save them in lists.

In [ ]:
# tune D to tune the number of grid points Nr such that it corresponds to that of your simulation
D = 3
Nr = 128*2^D + 3 # the overal course graining

# create the directory where the plots are saved
out_dir = "./convergence_plots/"*par*"/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");
(its_bln, all_filenames_bln)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="bilinears_");
(its_quantum, all_filenames_quantum)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="quantum_");

# check the timesteps where data are saved
# for classical geometry data (called "data")
println("length(its)=",length(its))
println("its",its)

# for the bilinears
println("length(its_bln)=",length(its_bln))
println("its_bln=",its_bln)

# for the state vector that has all the quantum modes
# strictly speaking, they are not needed below and can be commented out
println("length(its_quantum)=",length(its_quantum))
println("its_quantum=",its_quantum)

Load the data for a given iteration (it).

In [ ]:
# it is the iteration you analyze and it is an index running within the lists printed above i.e. it=1, its[it]=0
it = 50

# it saves the iteration number that it points to, within its list
it_data = its[it]
# makes this iteration number a string
it_str_data  = lpad(it_data, 4, "0")

# similar to above, for the bilinears
it_bln = its_bln[it]
it_str_bln  = lpad(it_bln, 4, "0")

# similar to above, for the quantum modes
it_quantum = its_quantum[it]
it_str_quantum  = lpad(it_quantum, 4, "0")

# labels for the classical variables
v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
# loads the classical variables at the given it
v = h5read(your_dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
# give names to specific variables needed later
Π  = v[:,2]
Ψ  = v[:,3]
A  = v[:,4]
oA = 1.0./A
B  = v[:,5]
oB = 1.0./B
KB = v[:,9]
α  = v[:,11]

# load the bilinears
bln  = h5read(your_dir*"/data_$(Nr)/bilinears_$(it_str_bln).h5","bilinears")
# load the quantum modes
v_quantum = h5read(your_dir*"/data_$(Nr)/quantum_$(it_str_quantum).h5","v")

# if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
t_data    =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
t_bln     =  h5readattr(your_dir*"/data_$(Nr)/bilinears_$(it_str_bln).h5", "/")["time"]
t_quantum =  h5readattr(your_dir*"/data_$(Nr)/quantum_$(it_str_quantum).h5", "/")["time"]

# checks if time instant is the same for all types of data
@assert t_data ≈ t_bln ≈ t_quantum

Build the different components of the stress-energy tensor (with mixed up-down indices), classical and quantum contributions separately.

In [ ]:
# classical
ρ  = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .+ Ψ.^2.0)
jA = -(oA.^0.5).*oB.*Π.*Ψ
SA = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .+ Ψ.^2.0)
SB = 0.5.*oA.*((oB.^2.0).*(Π.^2.0) .- Ψ.^2.0)

#quantum
hbar = 1.0
c = 1.0

ρ_quantum = @. (hbar*c^2/(4.0*π))*((0.5/α^2)*bln[:,2] + (0.5/A)*bln[:,3] + (1/B)*bln[:,5] - 0.5*bln[:,1] );
jA_quantum = @. -(hbar*c^2/(4.0*π))*(1.0/α)*bln[:,4];
SA_quantum = @. (hbar*c^2/(4.0*π))*( (0.5/α^2)*bln[:,2] + (0.5/A)*bln[:,3] - (1/B)*bln[:,5] - 0.5*bln[:,1] );
SB_quantum = @. (hbar*c^2/(4.0*π))*( (0.5/α^2)*bln[:,2] -(0.5/A)*bln[:,3] - 0.5*bln[:,1] );

Build the stress-energy tensor with both indices down, as well as specific components of the the 4 dimensional metric tensor (with both indices down).

In [ ]:
T_tt = @. α^2*(ρ + real(ρ_quantum));
T_rr = @. A*(SA + real(SA_quantum));
T_θθ = @. B*r^2*(SB + real(SB_quantum));

g_tt = @. -α^2;
g_rr = A;
g_θθ = @. B*r^2;

Plots

In [ ]:
# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

plot(r[ri_min:ri_max], real.(ρ_quantum[ri_min:ri_max]), title = "t=$(t_bln)",
     label = L"\rho_{quantum}", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

plot(r[ri_min:ri_max], real.(SA_quantum[ri_min:ri_max]), title = "t=$(t_bln)",
     label = L"S_{A \,quantum}", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

plot(r[ri_min:ri_max], real.(SB_quantum[ri_min:ri_max]), title = "t=$(t_bln)",
     label = L"S_{B \,quantum}", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

plot(r[ri_min:ri_max], real.(jA_quantum[ri_min:ri_max]), title = "t=$(t_bln)",
     label = L"j_{A \,quantum}", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

#plot(r[ri_min:ri_max], tt[ri_min:ri_max], title = "t=$(t_bln)",
#     label = "tt diff", linewidth = 2, color = "royalblue", frame = true, wsize = (800,600))
plot(r[ri_min:ri_max], g_tt[ri_min:ri_max], title = "t=$(t_bln)",# the index that r start from: r[3]=0
ri_min = 3
# the index tht r stops (in the plot)
ri_max = 512

plot(r[ri_min:ri_max], real.(ρ_quantum[ri_min:ri_max]), title = "t=$(t_bln)",
     label = L"\rho_{quantum}", linewidth = 2, frame = true, wsize = (800,600))
     label = "g_tt", linewidth = 2, frame = true, wsize = (800,600))
plot!(r[ri_min:ri_max], T_tt[ri_min:ri_max], title = "t=$(t_bln)",
     label = "T_tt", linewidth = 2, frame = true, wsize = (800,600))
plot!(r[ri_min:ri_max], -(T_tt./(ρ + real(ρ_quantum)))[ri_min:ri_max], title = "t=$(t_bln)",
     label = "-T_tt/<ρ>", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
ri_min = 3
ri_max = 512

plot(r[ri_min:ri_max], g_rr[ri_min:ri_max], title = "t=$(t_bln)",
     label = "g_rr", linewidth = 2, frame = true, wsize = (800,600))
plot!(r[ri_min:ri_max], T_rr[ri_min:ri_max], title = "t=$(t_bln)",
     label = "T_rr", linewidth = 2, frame = true, wsize = (800,600))

In [ ]:
ri_min = 3
ri_max = 512

plot(r[ri_min:ri_max], g_θθ[ri_min:ri_max], title = "t=$(t_bln)",
     label = "g_θθ", linewidth = 2, frame = true, wsize = (800,600))
plot!(r[ri_min:ri_max], T_θθ[ri_min:ri_max], title = "t=$(t_bln)",
     label = "T_θθ", linewidth = 2, frame = true, wsize = (800,600))